**BFS**

Implementation with Visualization of (6x10) matrix at each step. Shows how search explores and finds the goal get the path. 

**Note:** If you want just the final answer then its at the last cell

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display

# creating dictionary that stores info abour each cell of the cliff walker maze as per the question
maze_dict=dict()

start_position = (6,1)
goal_position = (6,10)
fringe_bfs=[] # simple queue for implementing bfs
path_bfs=[] # returned path for bfs

# backtracking_dict dictionary will be later used to back track the path of traversal that actually led to finding goal
backtracking_dict={start_position:[]}

#DeathZone in red as per question
#Shallow region in white as per question
#Deep region in green as per question
def init_cliff_walker(start_pos, goal_pos):
  for i in range(1,7):
    for j in range(1,11):
      if i==6 and (j<=9 and j>=2):
        maze_dict[(i,j)] = [" ","Deathzone","unvisited"]
      elif (i>=3 and i<=5) and ((j<=4 and j>=2) or (j<=8 and j>=6)):
        maze_dict[(i,j)] = [" ","Deep","unvisited"]
      else:
        maze_dict[(i,j)] = [" ","Shallow","unvisited"]
  maze_dict[start_pos][0] = "START"
  maze_dict[goal_pos][0] = "GOAL"

def print_cliff_walker():
  row_lst = []
  for row_num in range(1,7):
    lst=[]
    for col_num in range(1,11):
      if (row_num,col_num) == start_position or (row_num,col_num) == goal_position or ((row_num,col_num) in path_bfs):
        lst.append(str(maze_dict[(row_num,col_num)][0]))
      elif maze_dict[(row_num,col_num)][2] == "unvisited":
        lst.append(5*'_')
      elif maze_dict[(row_num,col_num)][2] == "visited":
        lst.append("\u03BF")
    row_lst.append(lst)
  
  idx_lst=[str(i) for i in range(1,7)]
  col_lst=[str(i) for i in range(1,11)]

  data_arr = np.array(row_lst)
  df = pd.DataFrame(data_arr, index=idx_lst, columns=col_lst)
  df = df.style.apply(style_specific_cell, axis=None)
  return df

def style_specific_cell(x):

    font_typ = 'font-weight: bold; text-align: center;'
    bg_color = ''
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    for key in maze_dict.keys():
      df1.iloc[key[0]-1, key[1]-1] = font_typ
      if maze_dict[key][1] == "Deep":
        bg_color = 'background-color: lightgreen;'
      elif maze_dict[key][1] == "Deathzone":
        bg_color = 'background-color: red;'
      elif maze_dict[key][1] == "Shallow":
        bg_color = 'background-color: white;'
      df1.iloc[key[0]-1, key[1]-1] += bg_color

      if key in path_bfs:
        df1.iloc[key[0]-1, key[1]-1] += 'color: blue;'
      else:
        df1.iloc[key[0]-1, key[1]-1] += 'color: black;'
        
    return df1

def get_cell_expense(cell_type):
  if cell_type == "Deep":
    return 5
  elif cell_type == "Shallow":
    return 1

# checks for valid neighbors/children to explore as per environment conditions. 
# Here valid means maze boundary condtions to be in 6x10 matrix, not encountering red deathzone and not revisiting already visited node
def get_valid_neighbors(curr_pos):
  neighbors = [(curr_pos[0]+1,curr_pos[1]), (curr_pos[0],curr_pos[1]+1),(curr_pos[0]-1,curr_pos[1]), (curr_pos[0],curr_pos[1]-1)]
  valid=[]
  for i in neighbors:
    if(i[0]<7 and i[0]>0) and (i[1]>0 and i[1]<11) and maze_dict[i][2] == "unvisited" and maze_dict[i][1]!="Deathzone":
      valid.append(i)
  return valid

  
init_cliff_walker(start_position, goal_position)
print("Initiating empty maze for BFS")
display(print_cliff_walker())


def bfs():

  print("Fringe => "+ str(fringe_bfs))
  curr = fringe_bfs.pop(0)
  
  # ignores visited positions to avoid redundant operation
  while maze_dict[curr][2] == "visited":
    curr = fringe_bfs.pop(0)
  
  maze_dict[curr][2] = "visited"
  print("popped "+str(curr)+" and tracked below:")
  display(print_cliff_walker())
  if maze_dict[curr][0]=="GOAL":
    path_bfs.append(curr)
    backtracking_dict[tuple()]= [curr]
    return True
  
  #collecting valid neighbors to expand the BFS tree
  children = get_valid_neighbors(curr)
  

  #updating backtracking dictionary
  for child in children:
    if child in backtracking_dict:
      backtracking_dict[child].append(curr)
    else:
      backtracking_dict[child] = [curr]
  
  # adding new children/neighbors to fringe (queue)
  fringe_bfs.extend(children)

  # continues recursive call till fringe empty or until GOAL not found
  while fringe_bfs!=[]:  
    result = bfs()
    if result == True:
      return True


#adding root/start node to bfs-fringe to start bfs
fringe_bfs.append(start_position)
if bfs():
  print("BFS found GOAL")
else:
  print("BFS failed to find GOAL")

#using backtracking to get path
temp_key=(6,10)
while backtracking_dict[temp_key] != []:
  ans = backtracking_dict[temp_key][0]
  path_bfs.append(ans)
  temp_key = ans
path_bfs = [path_bfs[x] for x in range( len(path_bfs)-1,-1,-1)] # reversing list to go from START to GOAL
print("BFS path backtracked START to GOAL =>"+str(path_bfs))

bfs_path_cost=0 # gives path cost sum
for i in path_bfs:
  bfs_path_cost += get_cell_expense(maze_dict[i][1])
  if i == start_position or i == goal_position:
    maze_dict[i][0]+= "="+str(bfs_path_cost)
  else:
    maze_dict[i][0]=bfs_path_cost  

print("BFS Path Cost = "+str(bfs_path_cost))
print()
print("Output of explored areas with output cost at each step higlighted in blue color:")
display(print_cliff_walker())

Initiating empty maze for BFS


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
3,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
4,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
5,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


Fringe => [(6, 1)]
popped (6, 1) and tracked below:


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
3,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
4,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
5,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


Fringe => [(5, 1)]
popped (5, 1) and tracked below:


...

,1,2,3,4,5,6,7,8,9,10
1,ο,ο,ο,ο,ο,ο,_____,_____,_____,_____
2,ο,ο,ο,ο,ο,ο,ο,_____,_____,_____
3,ο,ο,ο,ο,ο,ο,ο,ο,_____,_____
4,ο,ο,ο,ο,ο,ο,ο,ο,ο,_____
5,ο,ο,ο,ο,ο,ο,ο,ο,ο,ο
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


Fringe => [(1, 6), (6, 10), (4, 10), (4, 10), (3, 9), (3, 9), (2, 8), (2, 8), (1, 7), (1, 7)]
popped (6, 10) and tracked below:


,1,2,3,4,5,6,7,8,9,10
1,ο,ο,ο,ο,ο,ο,_____,_____,_____,_____
2,ο,ο,ο,ο,ο,ο,ο,_____,_____,_____
3,ο,ο,ο,ο,ο,ο,ο,ο,_____,_____
4,ο,ο,ο,ο,ο,ο,ο,ο,ο,_____
5,ο,ο,ο,ο,ο,ο,ο,ο,ο,ο
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


BFS found GOAL
BFS path backtracked START to GOAL =>[(6, 1), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 10)]
BFS Path Cost = 36

Output of explored areas with output cost at each step higlighted in blue color:


,1,2,3,4,5,6,7,8,9,10
1,ο,ο,ο,ο,ο,ο,_____,_____,_____,_____
2,ο,ο,ο,ο,ο,ο,ο,_____,_____,_____
3,ο,ο,ο,ο,ο,ο,ο,ο,_____,_____
4,ο,ο,ο,ο,ο,ο,ο,ο,ο,_____
5,2,7,12,17,18,23,28,33,34,35
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=36


**Final answer of BFS below**

In [2]:
print("BFS path backtracked START to GOAL =>"+str(path_bfs))
print("BFS Path Cost = "+str(bfs_path_cost))
print()
print("Output of explored areas with output cost at each step higlighted in blue color:")
display(print_cliff_walker())

BFS path backtracked START to GOAL =>[(6, 1), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 10)]
BFS Path Cost = 36

Output of explored areas with output cost at each step higlighted in blue color:


,1,2,3,4,5,6,7,8,9,10
1,ο,ο,ο,ο,ο,ο,_____,_____,_____,_____
2,ο,ο,ο,ο,ο,ο,ο,_____,_____,_____
3,ο,ο,ο,ο,ο,ο,ο,ο,_____,_____
4,ο,ο,ο,ο,ο,ο,ο,ο,ο,_____
5,2,7,12,17,18,23,28,33,34,35
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=36
